In [1]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

import torchvision
from torchvision import datasets
from torchvision import transforms

In [2]:
DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 2
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [9]:
model = torchvision.models.efficientnet_b0(pretrained=True)
print(model.classifier[1])
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
print(model.classifier[1])

Linear(in_features=1280, out_features=1000, bias=True)
Linear(in_features=1280, out_features=2, bias=True)


/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# tirar essa parte, porque não queremos testar redes diferentes

# def define_model(trial):
#     # We optimize the number of layers, hidden units and dropout ratio in each layer.
#     n_layers = trial.suggest_int("n_layers", 1, 3) # number of layers
#     layers = []

#     in_features = 1280
#     for i in range(n_layers):
#         out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128) # hidden units
#         layers.append(nn.Linear(in_features, out_features))
#         layers.append(nn.ReLU())
#         p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5) # dropout ratio
#         layers.append(nn.Dropout(p))

#         in_features = out_features
#     layers.append(nn.Linear(in_features, CLASSES))
#     layers.append(nn.LogSoftmax(dim=1))

#     return nn.Sequential(*layers)

In [11]:
from torchvision import transforms as T
img_size = 224

transform = T.Compose([
                T.Resize((img_size,img_size)),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

In [12]:
from torch.utils.data import DataLoader
from configs import Inputs
from utils.augmentations import get_transforms
from utils.data import FullRadiographSexDataset

val_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                       fold_nums=Inputs().val_folds,
                                       transforms=get_transforms(Inputs(), subset=["train"]))

val_dataloader = DataLoader(val_dataset,
                            batch_size=2,
                            shuffle=False,
                            num_workers=0)

train_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                         fold_nums=Inputs().train_folds,
                                         transforms=get_transforms(Inputs(), subset=["train"]))

train_dataloader = DataLoader(train_dataset,
                              batch_size=2,
                              shuffle=False,
                              num_workers=0)

Using only horizontal flip augmentation.
Using only horizontal flip augmentation.


In [15]:
def objective(trial):

    # Generate the model.
    #model = define_model(trial).to(DEVICE)
    model = torchvision.models.efficientnet_b0(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model = model.to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = train_dataloader, val_dataloader

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-12-13 16:37:17,584] A new study created in memory with name: no-name-2b1c7189-42f2-4acf-96c1-39444ab51fa6
[W 2022-12-13 16:37:17,876] Trial 0 failed because of the following error: RuntimeError('Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [2, 691200]')
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_10472/2906195414.py", line 28, in objective
    output = model(data)
  File "/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/torchvision/models/efficientnet.py", line 355, in forward
    return self._forward_impl(x)
  File "/opt/conda/lib/python3.8/site-packages/torchvision/models/efficientnet.py", line 345, in _forward_impl
    x = self.features(x)
  File "/opt/conda/lib/pyt

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [2, 691200]